## Best New Music
### Notebook 1
##### Webscrapes the 'best new music' from pitchfork and saves to .csv
##### Jack Collins

In [1]:
# Imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import Image

In [2]:
request = requests.get('https://pitchfork.com/best/')
print(f"Response code: {request.status_code}")

Response code: 200


In [3]:
request.text[:1000]

'<!DOCTYPE html><html lang="en"><head><title data-react-helmet="true">Best New Music: Tracks, Albums &amp; Reissues | Pitchfork</title><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no"/><meta data-react-helmet="true" name="og:type" content="website"/><meta data-react-helmet="true" name="og:site_name" content="Pitchfork"/><meta data-react-helmet="true" name="og:title" content="Pitchfork"/><meta data-react-helmet="true" name="og:url" content="https://pitchfork.com"/><meta data-react-helmet="true" name="description" content="The most exciting and important music being released today"/><meta data-react-helmet="true" name="og:description" content="The most exciting and important music being released today"/><script async="" src="/fonts-css/load-fonts.min.js"></script><link data-react-helmet="true" rel="shortcut icon" type="image/png" href="https://cdn.pitchfork.com/assets/misc/favicon-32.png"/><link data-react-helmet="true" rel=

In [4]:
# Convert the raw text into a BS object:
best = BeautifulSoup(request.text, 'lxml')
# All albums and tracks have the following class tags:
class_tags = ['bnm-hero__review-block', 'bnm-small album-small', 'bnm-small track-small']
# List of all the entries, looping through to get all information
reviews = best.find_all('div', {'class' : class_tags})

In [5]:
len(reviews)

21

#### Data format:
* Artist
* Title
* Preview of review
* Writer
* Rating
* Link to Pitchfork’s full review
* Album or song
* Genre
* Artwork

In [6]:
reviews[1]

<div class="bnm-hero__review-block"><a class="bnm-hero__link-block" href="/reviews/tracks/dijon-coogie/"><div class="artwork bnm-hero__artwork bnm-hero__artwork--with-notch"><img alt="Dijon: “coogie”" class="bnm-hero__img" src="https://media.pitchfork.com/photos/640105b25ec95150c5526999/1:1/w_320/Dijon%20Coogie%20Single%20Artwork.jpg"/></div><div class="details"><h3 class="bnm-hero__artist"><ul class="artist-list"><li>Dijon</li></ul></h3><h3 class="bnm-hero__title">“Coogie”</h3></div></a><ul class="genre-list genre-list--inline bnm-hero__genre-list"><li class="genre-list__item"><a class="genre-list__link" href="/reviews/tracks/?genre=pop">Pop/R&amp;B</a></li></ul><ul class="authors"><li><a class="linked display-name display-name--linked" href="/staff/jordan-coley/"><span class="by">by: </span>Jordan Coley</a></li></ul><p class="bnm-hero__date"><time class="pub-date" datetime="2023-03-03T19:50:49" title="Fri, 03 Mar 2023 19:50:49 GMT">12 hrs ago</time></p></div>

In [7]:
artwork = reviews[0].find('img').attrs['src']
print("Current best new album:")
Image(url=artwork)

Current best new album:


In [8]:
# Artist
print(reviews[0].find_all(['h3'])[0].text)
# Title
print(reviews[0].find_all(['h3'])[1].text)
# Link to review
print(reviews[0].find('a').attrs['href'])
# Album or track
print(reviews[0].find('a').attrs['href'].split('/')[2][:-1].title())
# Genre
print([genre.text for genre in reviews[0].find_all('li', {'class' : 'genre-list__item'})][0])
# Artwork
print(reviews[0].find('img').attrs['src'])

Kali Uchis
Red Moon in Venus
/reviews/albums/kali-uchis-red-moon-in-venus/
Album
Pop/R&B
https://media.pitchfork.com/photos/63cece0fa5957e2471744d13/1:1/w_320/Kali-Uchis-Red-Moon-in-Venus.jpg


In [9]:
[i.text for i in reviews[5].find_all(['ul'], {'class' : 'artist-list'})][0]

'Caroline Polachek'

In [10]:
# Artist
print(', '.join([i.text for i in reviews[5].find_all(['ul'], {'class' : 'artist-list'})]))
# Title
print(reviews[5].find(['h2']).text)
# Link to review
print(reviews[5].find('a').attrs['href'])
# Album or track
print(reviews[5].find('a').attrs['href'].split('/')[2][:-1].title())
# Genre
print([genre.text for genre in reviews[5].find_all('li', {'class' : 'genre-list__item'})][0])
# Artwork
print(reviews[5].find('img').attrs['src'])

Caroline Polachek
Desire, I Want to Turn Into You
/reviews/albums/caroline-polachek-desire-i-want-to-turn-into-you/
Album
Pop/R&B
https://media.pitchfork.com/photos/638e25b9050dcbdc83fd7926/1:1/w_160/Caroline-Polachek.jpg


In [11]:
# For other reviews
rev_request = requests.get('https://pitchfork.com' + reviews[0].find('a').attrs['href'])
rev_request.status_code

200

#### Function grab the best albums:

In [12]:
def get_pitchfork(URL):
    res = requests.get(URL)
    if res.status_code == 200:
        best = BeautifulSoup(res.text, 'lxml')
        class_tags = ['bnm-hero__review-block', 'bnm-small album-small', 'bnm-small track-small']
        reviews = best.find_all('div', {'class' : class_tags})
        all_dict = []
        for i, review in enumerate(reviews):
            if i <= 2:
                entry = {
                    'artist': review.find_all(['h3'])[0].text,
                    'title': review.find_all(['h3'])[1].text,
                    'review_link': URL + review.find('a').attrs['href'],
                    'album_track': review.find('a').attrs['href'].split('/')[2][:-1].title(),
                    'genre': [genre.text for genre in review.find_all('li', {'class' : 'genre-list__item'})],
                    'artwork': review.find('img').attrs['src'],
                }
            else:
                entry = {
                    'artist': ', '.join([i.text for i in review.find_all(['ul'], {'class' : 'artist-list'})]),
                    'title': review.find(['h2']).text,
                    'review_link': review.find('a').attrs['href'],
                    'album_track': review.find('a').attrs['href'].split('/')[2][:-1].title(),
                    'genre': [genre.text for genre in review.find_all('li', {'class' : 'genre-list__item'})][0],
                    'artwork': review.find('img').attrs['src'],
                }
            rev_res = requests.get(URL + review.find('a').attrs['href'])
            if rev_res.status_code == 200:
                rev = BeautifulSoup(rev_res.text, 'lxml')
                if entry['album_track'] == 'Album':
                    entry['preview'] = rev.find('div', {'class': 'review-detail__abstract'}).find('p').text
                    entry['score'] = rev.find('span', {'class': 'score'}).text
                else:
                    entry['preview'] = ''.join(rev.find('div', {'class' : 'contents'}).find('p').text.split('.')[:2])
                    entry['score'] = 'N/A'
                entry['author'] = rev.find('a', {'class': 'authors-detail__display-name'}).text
            all_dict.append(entry)
    new_music = pd.DataFrame(all_dict)
    new_music = new_music[new_music['album_track'] != 'Track']
    return new_music

In [13]:
best_new_music = get_pitchfork('https://pitchfork.com/best/')

In [14]:
best_new_music

,artist,title,review_link,album_track,genre,artwork
0,Kali Uchis,Red Moon in Venus,https://pitchfork.com/best//reviews/albums/kal...,Album,[Pop/R&B],https://media.pitchfork.com/photos/63cece0fa59...
2,Various Artists,Soul’d Out: The Complete Wattstax Collection,https://pitchfork.com/best//reviews/albums/var...,Album,[],https://media.pitchfork.com/photos/63ff7859f53...
3,Model/Actriz,Dogsbody,/reviews/albums/model-actriz-dogsbody/,Album,Rock,https://media.pitchfork.com/photos/63c562f0688...
4,Liv.e,Girl in the Half Pearl,/reviews/albums/live-girl-in-the-half-pearl/,Album,Pop/R&B,https://media.pitchfork.com/photos/63b85a20198...
5,Caroline Polachek,"Desire, I Want to Turn Into You",/reviews/albums/caroline-polachek-desire-i-wan...,Album,Pop/R&B,https://media.pitchfork.com/photos/638e25b9050...
6,Kelela,Raven,/reviews/albums/kelela-raven/,Album,Pop/R&B,https://media.pitchfork.com/photos/6373a6527e5...
7,Yo La Tengo,This Stupid World,/reviews/albums/yo-la-tengo-this-stupid-world/,Album,Rock,https://media.pitchfork.com/photos/636277ecdc5...
8,파란노을 (Parannoul),After the Magic,/reviews/albums/parannoul-after-the-magic/,Album,Rock,https://media.pitchfork.com/photos/63d4551997f...
15,Phew,Our Likeness,/reviews/albums/phew-our-likeness/,Album,Experimental,https://media.pitchfork.com/photos/63f4ebad1fb...
16,Laraaji,Segue to Infinity,/reviews/albums/laraaji-segue-to-infinity/,Album,Experimental,https://media.pitchfork.com/photos/63768f06f1e...


In [15]:
best_new_music.to_csv('new_music.csv', index=False) 

#### Notebook ends here. New music csv generated and can be used in the Spotify notebook